In [1]:
import pandas as pd
import glob
import os

In [14]:
# Caminho da pasta onde estão os arquivos CSV
caminho = r"D:\Downloads\csvs\*.csv"

# Lista todos os arquivos CSV da pasta
arquivos = glob.glob(caminho)

# Colunas de interesse (iguais em todos os arquivos)
colunas_uteis = [
    'DATA (YYYY-MM-DD)',
    'HORA (UTC)',
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
    'UMIDADE RELATIVA DO AR, HORARIA (%)',
    'RADIACAO GLOBAL (KJ/m²)',
    'VENTO, VELOCIDADE HORARIA (m/s)'
]

# Lista de DataFrames
dfs = []

for arquivo in arquivos:
    try:
        # Lê CSV
        df = pd.read_csv(arquivo, encoding='latin1', delimiter=';', skiprows=8, low_memory=False)

        # Mantém apenas colunas desejadas que existem no arquivo
        colunas_existentes = [col for col in colunas_uteis if col in df.columns]
        df = df[colunas_existentes]

        # Renomeia para nomes curtos e consistentes
        renomear = {
            'DATA (YYYY-MM-DD)': 'data',
            'HORA (UTC)': 'hora',
            'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'chuva',
            'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'temp',
            'UMIDADE RELATIVA DO AR, HORARIA (%)': 'umidade',
            'RADIACAO GLOBAL (KJ/m²)': 'radiacao',
            'VENTO, VELOCIDADE HORARIA (m/s)': 'vento'
        }
        df.rename(columns=renomear, inplace=True)

        # Combina data + hora (se ambas existirem)
        if 'data' in df.columns and 'hora' in df.columns:
            df['datahora'] = pd.to_datetime(df['data'] + ' ' + df['hora'], errors='coerce')

        # Converte colunas numéricas
        for col in ['chuva', 'temp', 'radiacao', 'vento']:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')

        # Adiciona nome do arquivo
        df['arquivo_origem'] = os.path.basename(arquivo)

        # Adiciona ao conjunto final
        dfs.append(df)

    except Exception as e:
        print(f"Erro ao ler {arquivo}: {e}")

# Junta todos os DataFrames
df_total = pd.concat(dfs, ignore_index=True)

# Reorganiza colunas
colunas_final = ['datahora', 'chuva', 'temp', 'umidade', 'radiacao', 'vento', 'arquivo_origem']
df_total = df_total[[col for col in colunas_final if col in df_total.columns]]

# Exibe amostra
print(df_total.head())


             datahora  chuva  temp  umidade  radiacao  vento  \
0 2007-01-01 00:00:00    0.0  22.4     89.0   -9999.0    3.3   
1 2007-01-01 01:00:00    0.0  22.2     90.0   -9999.0    4.3   
2 2007-01-01 02:00:00    0.0  22.1     91.0   -9999.0    3.9   
3 2007-01-01 03:00:00    0.0  22.0     93.0   -9999.0    3.5   
4 2007-01-01 04:00:00    1.0  21.6     96.0   -9999.0    3.9   

                                      arquivo_origem  
0  INMET_NE_PB_A310_AREIA_01-01-2007_A_31-12-2007...  
1  INMET_NE_PB_A310_AREIA_01-01-2007_A_31-12-2007...  
2  INMET_NE_PB_A310_AREIA_01-01-2007_A_31-12-2007...  
3  INMET_NE_PB_A310_AREIA_01-01-2007_A_31-12-2007...  
4  INMET_NE_PB_A310_AREIA_01-01-2007_A_31-12-2007...  


In [16]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1416 entries, 0 to 1415
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   data            1416 non-null   object        
 1   hora            1416 non-null   object        
 2   chuva           1416 non-null   float64       
 3   temp            1416 non-null   float64       
 4   umidade         1416 non-null   int64         
 5   radiacao        1416 non-null   float64       
 6   vento           1416 non-null   float64       
 7   datahora        1416 non-null   datetime64[ns]
 8   arquivo_origem  1416 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(1), object(3)
memory usage: 99.7+ KB


,chuva,temp,umidade,radiacao,vento,datahora
count,1416.000000,1416.000000,1416.000000,1416.000000,1416.000000,1416
mean,-127.100282,-97.538912,-88.983051,-3659.256215,-1437.962641,2017-12-02 11:30:00
min,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,2017-11-03 00:00:00
25%,0.000000,26.400000,32.000000,-9999.000000,1.700000,2017-11-17 17:45:00
50%,0.000000,29.700000,45.000000,68.450000,2.700000,2017-12-02 11:30:00
75%,0.000000,33.400000,59.000000,1982.925000,3.600000,2017-12-17 05:15:00
max,3.000000,37.800000,85.000000,3983.900000,8.200000,2017-12-31 23:00:00
std,1120.564372,1123.926494,1156.228083,5975.561071,3513.529416,NaN


In [36]:
df.columns

Index(['data', 'hora', 'chuva', 'temp', 'umidade', 'radiacao', 'vento',
       'datahora', 'arquivo_origem'],
      dtype='object')

In [38]:
df_total.columns

Index(['datahora', 'chuva', 'temp', 'umidade', 'radiacao', 'vento',
       'arquivo_origem'],
      dtype='object')

In [20]:
filtro = df_total[df_total['datahora'].dt.date == pd.to_datetime("2018-03-15").date()]
print(filtro)


                  datahora  chuva  temp  umidade  radiacao  vento  \
98184  2018-03-15 00:00:00    0.0  22.1     92.0   -9999.0    2.7   
98185  2018-03-15 01:00:00    0.0  22.0     92.0   -9999.0    3.4   
98186  2018-03-15 02:00:00    0.0  21.7     93.0   -9999.0    2.1   
98187  2018-03-15 03:00:00    0.0  21.4     94.0   -9999.0    1.9   
98188  2018-03-15 04:00:00    0.0  21.1     94.0   -9999.0    1.9   
...                    ...    ...   ...      ...       ...    ...   
860155 2018-03-15 19:00:00    0.0  32.8     43.0    1665.4    1.8   
860156 2018-03-15 20:00:00    0.0  32.4     45.0     990.4    2.7   
860157 2018-03-15 21:00:00    0.0  30.6     51.0     148.9    2.2   
860158 2018-03-15 22:00:00    0.0  29.3     55.0   -9999.0    1.0   
860159 2018-03-15 23:00:00    0.0  27.2     65.0   -9999.0    1.5   

                                           arquivo_origem  
98184   INMET_NE_PB_A310_AREIA_01-01-2018_A_31-12-2018...  
98185   INMET_NE_PB_A310_AREIA_01-01-2018_A_31-12-2

In [9]:
print(df_total.tail(50))

                  datahora  chuva  temp  umidade  radiacao   vento  \
798430 2017-12-29 22:00:00    0.0  34.0     32.0   -9999.0 -9999.0   
798431 2017-12-29 23:00:00    0.0  33.7     32.0   -9999.0 -9999.0   
798432 2017-12-30 00:00:00    0.0  33.1     34.0   -9999.0 -9999.0   
798433 2017-12-30 01:00:00    0.0  32.1     42.0   -9999.0 -9999.0   
798434 2017-12-30 02:00:00    0.0  30.2     52.0   -9999.0 -9999.0   
798435 2017-12-30 03:00:00    0.0  29.0     56.0   -9999.0 -9999.0   
798436 2017-12-30 04:00:00    0.0  28.1     54.0   -9999.0 -9999.0   
798437 2017-12-30 05:00:00    0.0  27.3     58.0   -9999.0 -9999.0   
798438 2017-12-30 06:00:00    0.0  26.3     60.0   -9999.0 -9999.0   
798439 2017-12-30 07:00:00    0.0  25.5     63.0   -9999.0 -9999.0   
798440 2017-12-30 08:00:00    0.0  24.6     68.0   -9999.0 -9999.0   
798441 2017-12-30 09:00:00    0.0  24.8     67.0      94.3 -9999.0   
798442 2017-12-30 10:00:00    0.0  26.5     60.0     779.5 -9999.0   
798443 2017-12-30 11

In [28]:
filtro = df_total[df_total['datahora'].dt.date == pd.to_datetime("2015-03-15").date()]
print(filtro)


                  datahora   chuva    temp  umidade  radiacao   vento  \
71880  2015-03-15 00:00:00 -9999.0 -9999.0  -9999.0   -9999.0 -9999.0   
71881  2015-03-15 01:00:00 -9999.0 -9999.0  -9999.0   -9999.0 -9999.0   
71882  2015-03-15 02:00:00 -9999.0 -9999.0  -9999.0   -9999.0 -9999.0   
71883  2015-03-15 03:00:00 -9999.0 -9999.0  -9999.0   -9999.0 -9999.0   
71884  2015-03-15 04:00:00 -9999.0 -9999.0  -9999.0   -9999.0 -9999.0   
...                    ...     ...     ...      ...       ...     ...   
766723 2015-03-15 19:00:00     0.0    29.0     67.0    1445.9     5.4   
766724 2015-03-15 20:00:00     0.0    27.5     73.0     649.9     5.2   
766725 2015-03-15 21:00:00     0.0    26.4     80.0      39.6     0.5   
766726 2015-03-15 22:00:00     0.0    25.8     83.0   -9999.0     0.0   
766727 2015-03-15 23:00:00     0.0    25.6     85.0   -9999.0     0.4   

                                           arquivo_origem  
71880   INMET_NE_PB_A310_AREIA_01-01-2015_A_31-12-2015...  
718

In [50]:
# Caminho da pasta onde estão os arquivos CSV
caminho = r"D:\Downloads\csvs2\*.csv"

# Lista todos os arquivos CSV da pasta
arquivos = glob.glob(caminho)

# Colunas de interesse (iguais em todos os arquivos)
colunas_uteis = [
    'Data',
    'Hora UTC',
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
    'UMIDADE RELATIVA DO AR, HORARIA (%)',
    'RADIACAO GLOBAL (KJ/m²)',
    'VENTO, VELOCIDADE HORARIA (m/s)'
]

# Lista de DataFrames
dfs = []

for arquivo in arquivos:
    try:
        # Lê CSV
        df2 = pd.read_csv(arquivo, encoding='latin1', delimiter=';', skiprows=8, low_memory=False)

        # Verifique as colunas do arquivo
        print(f"Colunas em {arquivo}: {df2.columns.tolist()}")

        # Mantém apenas colunas desejadas que existem no arquivo
        colunas_existentes = [col for col in colunas_uteis if col in df2.columns]
        df2 = df2[colunas_existentes]

        # Renomeia para nomes curtos e consistentes
        renomear = {
            'Data': 'data',
            'Hora UTC': 'hora',
            'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'chuva',
            'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'temp',
            'UMIDADE RELATIVA DO AR, HORARIA (%)': 'umidade',
            'RADIACAO GLOBAL (KJ/m²)': 'radiacao',
            'VENTO, VELOCIDADE HORARIA (m/s)': 'vento'
        }
        df2.rename(columns=renomear, inplace=True)

        # Verifica se as colunas 'data' e 'hora' existem e tem dados
        if 'data' in df2.columns and 'hora' in df2.columns:
            # Diagnóstico: Verifique as primeiras linhas de 'data' e 'hora' antes da conversão
            print(f"Primeiras linhas de 'data' e 'hora' no arquivo {arquivo}:")
            print(df2[['data', 'hora']].head())

            # Combine data + hora
            df2['datahora'] = pd.to_datetime(df2['data'].astype(str) + ' ' + df2['hora'].astype(str), errors='coerce')

            # Diagnóstico: Verifique as primeiras linhas após combinar
            print(f"Primeiras linhas de 'datahora' no arquivo {arquivo}:")
            print(df2[['datahora']].head())

        # Converte colunas numéricas
        for col in ['chuva', 'temp', 'radiacao', 'vento']:
            if col in df2.columns:
                # Substitui vírgulas por ponto e converte para numérico
                df2[col] = pd.to_numeric(df2[col].astype(str).str.replace(',', '.'), errors='coerce')

        # Adiciona nome do arquivo
        df2['arquivo_origem'] = os.path.basename(arquivo)

        # Adiciona ao conjunto final
        dfs.append(df2)

    except Exception as e:
        print(f"Erro ao ler {arquivo}: {e}")

# Junta todos os DataFrames
df_total2018 = pd.concat(dfs, ignore_index=True)

# Reorganiza colunas
colunas_final = ['datahora', 'chuva', 'temp', 'umidade', 'radiacao', 'vento', 'arquivo_origem']
df_total2018 = df_total2018[[col for col in colunas_final if col in df_total2018.columns]]

# Exibe amostra
print(df_total2018.head())


Colunas em D:\Downloads\csvs2\INMET_NE_PB_A310_AREIA_01-01-2019_A_31-12-2019.CSV: ['Data', 'Hora UTC', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)', 'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)', 'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)', 'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)', 'RADIACAO GLOBAL (KJ/m²)', 'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)', 'TEMPERATURA DO PONTO DE ORVALHO (°C)', 'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)', 'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)', 'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)', 'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)', 'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)', 'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)', 'UMIDADE RELATIVA DO AR, HORARIA (%)', 'VENTO, DIREÇÃO HORARIA (gr) (° (gr))', 'VENTO, RAJADA MAXIMA (m/s)', 'VENTO, VELOCIDADE HORARIA (m/s)', 'Unnamed: 19']
Primeiras linhas de 'data' e 'hora' no arquivo D:\Downloads\csvs2\INMET_NE_PB_A310_AREIA_01-01-2019_A_31-12-2019.CSV:
         data      hora
0

In [52]:
print(df_total2018.head(50))
df_total2018.columns

                    datahora  chuva  temp  umidade  radiacao  vento  \
0  2019-01-01 00:00:00+00:00    0.0  22.3     88.0      16.3    4.8   
1  2019-01-01 01:00:00+00:00    0.0  22.2     90.0      26.1    4.7   
2  2019-01-01 02:00:00+00:00    0.0  22.1     91.0      30.5    4.1   
3  2019-01-01 03:00:00+00:00    0.0  21.9     92.0      27.8    4.1   
4  2019-01-01 04:00:00+00:00    0.0  21.8     93.0      29.4    4.0   
5  2019-01-01 05:00:00+00:00    0.0  21.8     93.0      29.8    4.1   
6  2019-01-01 06:00:00+00:00    0.0  21.7     93.0      27.5    4.5   
7  2019-01-01 07:00:00+00:00    0.0  21.7     93.0      31.5    3.9   
8  2019-01-01 08:00:00+00:00    0.0  21.4     94.0      29.6    4.7   
9  2019-01-01 09:00:00+00:00    0.0  21.5     94.0      88.3    5.9   
10 2019-01-01 10:00:00+00:00    0.0  23.1     88.0     429.9    4.7   
11 2019-01-01 11:00:00+00:00    0.0  24.2     75.0    1327.5    5.2   
12 2019-01-01 12:00:00+00:00    0.0  25.8     67.0    1647.3    4.8   
13 201

Index(['datahora', 'chuva', 'temp', 'umidade', 'radiacao', 'vento',
       'arquivo_origem'],
      dtype='object')

In [56]:
df_total2018.columns

Index(['datahora', 'chuva', 'temp', 'umidade', 'radiacao', 'vento',
       'arquivo_origem'],
      dtype='object')

In [58]:
df_total.columns

Index(['datahora', 'chuva', 'temp', 'umidade', 'radiacao', 'vento',
       'arquivo_origem'],
      dtype='object')

In [60]:
df_final = pd.concat([df_total, df_total2018], ignore_index=True)

In [1]:
df_final.head(100)

NameError: name 'df_final' is not defined

In [64]:
df_final.tail()

,datahora,chuva,temp,umidade,radiacao,vento,arquivo_origem
1280635,2024-12-31 19:00:00+00:00,NaN,NaN,NaN,NaN,NaN,INMET_NE_PB_A373_ITAPORANGA_01-01-2024_A_31-12...
1280636,2024-12-31 20:00:00+00:00,NaN,NaN,NaN,NaN,NaN,INMET_NE_PB_A373_ITAPORANGA_01-01-2024_A_31-12...
1280637,2024-12-31 21:00:00+00:00,NaN,NaN,NaN,NaN,NaN,INMET_NE_PB_A373_ITAPORANGA_01-01-2024_A_31-12...
1280638,2024-12-31 22:00:00+00:00,NaN,NaN,NaN,NaN,NaN,INMET_NE_PB_A373_ITAPORANGA_01-01-2024_A_31-12...
1280639,2024-12-31 23:00:00+00:00,NaN,NaN,NaN,NaN,NaN,INMET_NE_PB_A373_ITAPORANGA_01-01-2024_A_31-12...
